# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [19]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
import sys

#### Creating list of filepaths to process original event csv data files

In [20]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [21]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [22]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [23]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [24]:
#Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [25]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

In [26]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# IN apache casandra always think QUERY-FIRST .
# so we start by writing the query first.


## session id is unique by itself so when filtered by it will give a vaild filtering to QUERY_1, moreover,  QUERY_1 asks for the certain info above and during session id and item_in_session
## therefore, it is clustered by item_in_session . 
QUERY_1 = """SELECT artist_name, song_title, song_length
FROM music_app 
WHERE session_id = 338 
AND item_in_Session = 4 
"""

                    

In [27]:
# we create the table based on the query we write above .

##music_app table session id is filtered by session id, and clustered by item_in_session . 
query = """CREATE TABLE IF NOT EXISTS music_app(
session_id int ,
item_in_session int,
artist_name text,
song_title text,
song_length double,
PRIMARY KEY(session_id,item_in_session)
)"""

try: 
    session.execute(query)
except Exception as e:
    print(e)
    

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [28]:
# we create a dataframe with a constant chunksize incase we deal with BIG DATA so thaat we dont ran out of RAM space .
df = pd.read_csv("event_datafile_new.csv", chunksize=500)

insert_query ="INSERT INTO music_app (session_id, item_in_session, artist_name, song_title, song_length ) VALUES (%s,%s,%s,%s,%s)"

for i , df in enumerate(df):
    for j, row in df.iterrows():
        try:
            session.execute(insert_query,(row.sessionId ,row.itemInSession, row.artist,row.song, row.length ) )
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [29]:
## Add in the SELECT statement to verify the data was entered into the table

try:
    rows = session.execute(QUERY_1)
except Exception as e:
    print(e)

In [30]:
for row in rows:
    print(row)

Row(artist_name='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.3073)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [31]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

## Think query first .
## user_id and session_id when combined give a Unique value , thus, we filter our table using the composite partition key ((user_id,session+id))
QUERY_2= """SELECT artist_name , user_first_name, user_last_name ,song_title 
FROM sorted_songs 
WHERE user_id = 10
AND session_id = 182 """ 

## a composite partition key to filter the created table which is made of user id & session id. Moreover, query 2 wants the song title sorted by 
## item_in_session column, therefore item_in_session is selected to be the clustering key . 
query = """CREATE TABLE IF NOT EXISTS sorted_songs(
user_id int , 
session_id int ,
item_in_session int ,
artist_name text,
song_title text,
user_first_name text,
user_last_name text,
PRIMARY KEY((user_id,session_id) , item_in_session)
)"""

try: 
    session.execute(query)
except Exception as e:
    print(e)
    
df = pd.read_csv("event_datafile_new.csv", chunksize=1000)

insert_query ="INSERT INTO sorted_songs (user_id, session_id, item_in_session, artist_name, song_title, user_first_name, user_last_name) VALUES (%s,%s,%s,%s,%s,%s,%s)"

for i , df in enumerate(df):
    for j, row in df.iterrows():
        try:
            session.execute(insert_query,(row.userId ,row.sessionId ,row.itemInSession, row.artist,row.song, row.firstName, row.lastName) )
        except Exception as e:
            print(e)
                    

In [32]:

#Add in the SELECT statement to verify the data was entered into the table .
try:
    rows = session.execute(QUERY_2)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist_name='Down To The Bone', user_first_name='Sylvie', user_last_name='Cruz', song_title="Keep On Keepin' On")
Row(artist_name='Three Drives', user_first_name='Sylvie', user_last_name='Cruz', song_title='Greece 2000')
Row(artist_name='Sebastien Tellier', user_first_name='Sylvie', user_last_name='Cruz', song_title='Kilometer')
Row(artist_name='Lonnie Gordon', user_first_name='Sylvie', user_last_name='Cruz', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)')


In [33]:
##  Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


## song_title will be the partition key therefore it's going to be filtered by the song_title
## because this table is created to get users who listened to a certain song. 
QUERY_3= """SELECT  user_first_name, user_last_name  
FROM user_listen_songs 
WHERE song_title = 'All Hands Against His Own'
"""

## Since we want to get users who listen the song "all hands agains his own" we filter the user_listen_songs tables by
## song_title column, clustered (and sorted) by user_id column .
query = """CREATE TABLE IF NOT EXISTS user_listen_songs(
song_title text,
user_id int ,
user_first_name text,
user_last_name text,
 
PRIMARY KEY(song_title,user_id)
)"""

try: 
    session.execute(query)
except Exception as e:
    print(e)
    
df = pd.read_csv("event_datafile_new.csv", chunksize=500)

insert_query ="INSERT INTO user_listen_songs (song_title,user_id, user_first_name, user_last_name) VALUES (%s,%s,%s,%s)"

for i , df in enumerate(df):
    for j, row in df.iterrows():
        try:
            session.execute(insert_query,(row.song,row.userId, row.firstName, row.lastName) )
        except Exception as e:
            print(e)

                    

In [34]:
#verify .
try:
    rows = session.execute(QUERY_3)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(user_first_name='Jacqueline', user_last_name='Lynch')
Row(user_first_name='Tegan', user_last_name='Levine')
Row(user_first_name='Sara', user_last_name='Johnson')


### Drop the tables before closing out the sessions

In [35]:
#Drop the table before closing out the sessions
drops = ["DROP TABLE IF EXISTS music_app","DROP TABLE IF EXISTS sorted_songs","DROP TABLE IF EXISTS user_listen_songs"]
for drop in drops:
    session.execute(drop)

### Close the session and cluster connection¶

In [36]:
session.shutdown()
cluster.shutdown()